In [4]:
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import openai
from urllib.request import urlopen
import json

In [16]:
df1 = pd.read_csv('df1_for_GPT3_search.csv')

In [17]:
df1.head()

,keyPhrases,paperAbstract,numKeyReferences,title,venue,numCitedBy,numKeyCitations,docno
0,"['Isomorphism', 'Duality', 'DUAL', 'Undirected...","[""We provide a correspondence between the subj...",[1],['Duality Theorems for Finite Structures (Char...,['JCT'],[54],[4],6e4eddf4d6671c37537bb5d1c9623353b62e8531
1,"['Person Reidentification', 'China Email', 'Si...",['—This paper proposes a novel approach to per...,[4],['Deep Ranking for Person Re-identification vi...,['TIP'],[6],[0],8d8afef13a8f6195d3b874231e5e767cf62f3c50
2,"['Natural Language Text', 'PML', 'C.F.', 'Auto...",['We present a system for identifying the sema...,[1],['Automatic Labeling of Semantic Roles'],['COLING'],[684],[63],04a8d271bc4384dbfbb417bfb625feb01cb44666
3,"['Motion SFM', 'Landmarkbased Approaches', 'Gr...",['Ego-motion estimation for an agile single ca...,[1],['Real-Time Simultaneous Localisation and Mapp...,['ICCV'],[483],[35],102e50c2c9a8e3d00de64a26759916c926fa3db6
4,"['Semantic Pattern', 'Positive Sentiment', 'Se...",['Sentiment analysis over Twitter offer organi...,[0],['Semantic Sentiment Analysis of Twitter'],['SEMWEB'],[57],[1],ec4a94637ecd115219869e9df8902cb7282481e0


In [6]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

In [3]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [19]:
doc_embeddings = np.load("title_ab_embeddings_gpt3.npy", allow_pickle=True)

In [20]:
df1['doc_emb'] = doc_embeddings

In [72]:
def search(query):
    top100 = search_100(query)
    result = rerank(df1, query, 10, top100)
    dict_list = []
    for index, row in result.iterrows():
        temp = {}
        temp['title'] = row['title']
        temp['paperAbstract'] = row['paperAbstract']
        temp['venue'] = row['venue']
        temp['numCitedBy'] = row['numCitedBy']
        dict_list.append(temp)
    return dict_list

In [18]:
def search_100(query):
    MAX_SIZE = 100
    payload = {
        "multi_match": {
                "query": query,
                "fields": ['paperAbstract','title']
            }
            
        }
    resp = es.search(index="s2_doc", query=payload, size=MAX_SIZE)
    result = []
    # Extract info needed for display. 
    for item in resp['hits']['hits']:
        result.append(item['_id'])
    return result

In [66]:
def rerank(df, query, n, top100):
    mask = df['docno'].isin(top100)
    selected_rows = df.loc[mask]
    column_name = "similarity"
    query = openai.Embedding.create(
    input=query,
    engine="text-search-davinci-query-001"
)["data"][0]["embedding"]
    selected_rows[column_name] = selected_rows['doc_emb'].apply(lambda x: cosine_similarity(x["data"][0]["embedding"], query))
    result = (
        selected_rows.sort_values(column_name, ascending=False)
        .head(n)
    )
    result = result.drop(columns = ['docno', 'doc_emb'])
#     result = pd.DataFrame.to_json(result, orient='index')
#     result = json.loads(result)
    return result

In [36]:
import time
start = time.time()
result = search("information retrieval")
end = time.time()
diff = end - start
print(diff)
result

/var/folders/_y/xx88m0sx4s96c12lqts9h7tm0000gn/T/ipykernel_92328/3578028534.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows[column_name] = selected_rows['doc_emb'].apply(lambda x: cosine_similarity(x["data"][0]["embedding"], query))


2.384714126586914


,keyPhrases,paperAbstract,numKeyReferences,title,venue,numCitedBy,numKeyCitations,docno,doc_emb,similarity
5577,"['Translation Language Model', 'Formule', 'Voc...",['We propose a new probabilistic approach to i...,[0],['Information Retrieval as Statistical Transla...,['SIGIR'],[294],[26],15281680463698dca403697bd627af4efebc98a2,"{'object': 'list', 'data': [{'object': 'embedd...",0.403286
6324,"['TREC', 'Statistical Significance', 'Average ...",['The effectiveness of information retrieval s...,[3],['Information retrieval system evaluation: eff...,['SIGIR'],[166],[6],4314bbc2a62c37a9b66d759fc35ae6f7607344e0,"{'object': 'list', 'data': [{'object': 'embedd...",0.397071
3615,"['Agglomerative Hierarchical Clustering', 'Que...",['1 Boolean retrieval 1 2 The term vocabulary ...,[0],['Introduction to information retrieval'],['DAGLIB'],[2835],[173],040678daf6a49a88345ee0c680fccfd134f24d4b,"{'object': 'list', 'data': [{'object': 'embedd...",0.393794
1406,"['Complex Number', 'Conditional Logic', 'Quant...","['Information retrieval, IR, the science of ex...",[0],['The geometry of information retrieval'],['DAGLIB'],[108],[4],f43f289efeadfe63c553876eeec71f97f5d8c1a5,"{'object': 'list', 'data': [{'object': 'embedd...",0.391709
7695,"['Language Model', 'IR Model', 'Query Term', '...",['Empirical studies of information retrieval m...,[0],['A formal study of information retrieval heur...,['SIGIR'],[119],[6],2c68c7faa89b104b78e2850dbade5a81f0743874,"{'object': 'list', 'data': [{'object': 'embedd...",0.387235
5567,NaN,[''],[0],['Information Retrieval - Algorithms and Heuri...,['IRS'],[162],[3],b1a660f7dec8ec270f52cf0bf2dc8ef98ae1dd11,"{'object': 'list', 'data': [{'object': 'embedd...",0.386882
446,"['WWW', 'ENT', 'Engine', 'Information Retrieva...",['In this paper we review studies of the growt...,[5],['Information retrieval on the web'],['CSUR'],[130],[6],20987764a34a506a3235446a75aeea0d4dc1b4e8,"{'object': 'list', 'data': [{'object': 'embedd...",0.386806
7880,NaN,[''],[0],['Information retrieval for music and motion'],['DAGLIB'],[226],[17],b9bcc3bdb022b47f7541acaa09c8d9ca0ae27a16,"{'object': 'list', 'data': [{'object': 'embedd...",0.386644
160,NaN,[''],[0],['Introduction to Modern Information Retrieval'],['MG'],[3472],[153],3bb73407096cbb0f5d884eb108d694b43580004b,"{'object': 'list', 'data': [{'object': 'embedd...",0.381862
1895,"['Average Precision', 'B Ttcher', 'Inverted In...",[''],[0],['Information Retrieval - Implementing and Eva...,['DAGLIB'],[104],[1],0a717685158723040c56b42d67fc8988fab9fc73,"{'object': 'list', 'data': [{'object': 'embedd...",0.381144


In [73]:
result = search("deep learning")
result

/var/folders/_y/xx88m0sx4s96c12lqts9h7tm0000gn/T/ipykernel_92328/886092020.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows[column_name] = selected_rows['doc_emb'].apply(lambda x: cosine_similarity(x["data"][0]["embedding"], query))


[{'title': "['Deep learning.']",
  'paperAbstract': "['Overview of the tutorial • A brief history of deep learning. • How to learn multi-layer generative models of unlabelled data by learning one layer of features at a time. – What is really going on when we stack RBMs to form a deep belief net. • How to use generative models to make discriminative training methods work much better for classification and regression. • How to modify RBMs to deal with real-valued input.']",
  'venue': "['Nature']",
  'numCitedBy': '[254]'},
 {'title': "['Deep Learning of Representations for Unsupervised and Transfer Learning']",
  'paperAbstract': "['Deep learning algorithms seek to exploit the unknown structure in the input distribution in order to discover good representations, often at multiple levels, with higher-level learned features defined in terms of lower-level features. The objective is to make these higher-level representations more abstract, with their individual features more invariant to m